In [208]:
# CREATED: 4-APR-2022
# LAST EDIT: 26-APR-2022
# AUTHOR(S): DUANE RINEHART
'''
IMPORT POLYGON POINTS FROM CSV (EVENTUALLY PULL FROM RDBMS)
Note: polygons_dump5.csv GENERATED WITH QUERY - SELECT * FROM annotations_points WHERE (prep_id='DK55' AND (label='7N_L' OR label='5N_L')) OR (prep_id='MD594' AND (label='SC' OR label='IC'))
'''
import os
import pandas as pd
import numpy as np

win_data_dir = 'E:/dev/'
linux_data_dir = '/mnt/e/dev/'


'''
SET SRC FOLDER BASED (CROSS-PLATFORM)
'''
if os.name == "nt":
    input_fullPath = os.path.join(win_data_dir)
else:
    input_fullPath = os.path.join(linux_data_dir)

infile = os.path.join(input_fullPath, "polygons_dump5.csv")


def load_raw_data(infile):
    return pd.read_csv(infile, sep='\t', index_col=0)


def parse_df(df_raw_data, prep_id, label, xy_resolution_pixels_microns, z_resolution_section_to_microns):
    data = df_raw_data[np.logical_and(df_raw_data.prep_id==prep_id,df_raw_data.label==label)]
    data['x'] = data['x'].apply(lambda x: x / xy_resolution_pixels_microns)
    data['y'] = data['y'].apply(lambda y: y / xy_resolution_pixels_microns)
    data['z'] = data['z'].apply(lambda z: z / z_resolution_section_to_microns)
    data["unique_key"] =  data["prep_id"].astype(str) + '_' + data["label"].astype(str) + '_' + data["z"].round().astype(int).astype(str)
    data["coord"] = list(zip(data['x'], data["y"], data["z"]))
    data[(data['prep_id'] == prep_id) & (data['label'] == label)]
    selected_columns = data[
        ["unique_key", "prep_id", "label", "coord", "z", "polygon_id", "volume_id", "ordering"]
    ]  # EXTRACT SPECIFIC COLUMNS FOR ANALYSIS
    return data


df_raw_data = load_raw_data(infile) # REPLACE W/ RDBMS CALL FOR PRODUCTION


# FILTERING CONSTANTS
z_resolution_section_to_microns = 20 # STORED IN DB

# POLYGON SETS BY prep_id, label CREATED INDIVIDUALLY BELOW FOR TESTING
prep_id = 'MD594'
xy_resolution_pixels_microns = .452 # STORED IN DB (FOR FOUNDATION BRAINS)
label = 'SC'
extracted_columns_MD594_SC = parse_df(df_raw_data, prep_id, label, xy_resolution_pixels_microns, z_resolution_section_to_microns)
label = 'IC'
extracted_columns_MD594_IC = parse_df(df_raw_data, prep_id, label, xy_resolution_pixels_microns, z_resolution_section_to_microns)
prep_id = 'DK55'
xy_resolution_pixels_microns = .325 # STORED IN DB (FOR DK55)
label = '7N_L'
extracted_columns_DK55_7N_L = parse_df(df_raw_data, prep_id, label, xy_resolution_pixels_microns, z_resolution_section_to_microns)
label = '7N_R'
extracted_columns_DK55_7N_R = parse_df(df_raw_data, prep_id, label, xy_resolution_pixels_microns, z_resolution_section_to_microns)

# EACH DATAFRAME CONTAINS LIST OF TUPLES (coord) THAT MAY BE USED FOR PLOTTING POLYGONS IN NEUROGLANCER (SOME EXAMPLES IN NOTEBOOK - BELOW)

# FOLLOWING CODE ONLY USEFUL IF BATCH PROCESSING ALL DATA (HOWEVER FOR INTEGRATION IN APP (PIPELINE), LIKELY DB QUERY WITH LOOP IS SUFFICIENT)

# consolidated_coordinates = pd.concat([extracted_columns_MD594_SC, extracted_columns_MD594_IC, extracted_columns_DK55_7N_L, extracted_columns_DK55_7N_R])
#
# # CREATE DICTIONARY OF COORDINATES
# transformed_polygon_structures = consolidated_coordinates.groupby(['unique_key'])['coord'].apply(lambda x: list(np.unique(x))).to_dict()
#
# # OUTPUT FORMAT:
# # -KEY IS CONCATENATED prep_id_structure_section ("_" delimiter)
# # -VALUE IS LIST OF COORDINATE TUPLES (x, y, z)
# #
# # TO ACCESS [EXAMPLE WITH prep_id 'MD594', label 'IC', section 140]:
# polygon_index = 'MD594_IC_140'
# for coordinate_list in transformed_polygon_structures[polygon_index]:
#     print(coordinate_list)

<ipython-input-208-1fb089f6a103>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['x'] = data['x'].apply(lambda x: x / xy_resolution_pixels_microns)
<ipython-input-208-1fb089f6a103>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['y'] = data['y'].apply(lambda y: y / xy_resolution_pixels_microns)
<ipython-input-208-1fb089f6a103>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

(26043.141592920354, 9466.615044247786, 140.0)
(26060.398230088493, 9397.057522123894, 140.0)
(26077.87610619469, 9327.5, 140.0)
(26095.353982300883, 9501.393805309734, 140.0)
(26112.610619469026, 9257.920353982301, 140.0)
(26130.08849557522, 9205.75221238938, 140.0)
(26130.08849557522, 9553.561946902655, 140.0)
(26164.82300884956, 9136.17256637168, 140.0)
(26164.82300884956, 9605.730088495575, 140.0)
(26182.079646017697, 9084.004424778761, 140.0)
(26199.557522123894, 9657.898230088495, 140.0)
(26217.035398230088, 9031.814159292035, 140.0)
(26234.51327433628, 9710.066371681416, 140.0)
(26269.026548672566, 8979.646017699115, 140.0)
(26303.982300884953, 9727.45575221239, 140.0)
(26321.2389380531, 8944.845132743363, 140.0)
(26338.716814159292, 9710.066371681416, 140.0)
(26355.97345132743, 8910.066371681416, 140.0)
(26390.929203539825, 9692.654867256637, 140.0)
(26408.185840707964, 8857.87610619469, 140.0)
(26460.398230088496, 8823.097345132743, 140.0)
(26460.398230088496, 9657.87610619469

In [217]:
# TO VIEW RANGE OF SECTION VALUES FOR BRAIN/STRUCTURE (STORED SEPARATELY)
extracted_columns_MD594_SC.z.unique()

array([161., 157., 248., 173., 164., 279., 209., 268., 274., 200., 263.,
       176., 146., 288., 296., 189., 196., 165., 212., 186., 160., 214.,
       217., 178., 287., 292., 202., 139., 218., 199., 163., 219., 242.,
       260., 142., 272., 188., 179., 175., 144., 152., 137., 205., 167.,
       251., 203., 235., 305., 285., 184., 195., 228., 266., 236., 239.,
       270., 282., 267., 241., 148., 306., 162., 197., 244., 291., 169.,
       293., 194., 280., 229., 230., 149., 143., 256., 177., 147., 180.,
       245., 150., 172., 231., 204., 304., 246., 275., 171., 249., 181.,
       207., 286., 201., 257., 281., 237., 259., 182., 262., 278., 250.,
       254., 158., 215., 232., 252., 225., 301., 307., 276., 233., 154.,
       289., 166., 309., 210., 269., 192., 255., 265., 308., 299., 220.,
       298., 243., 226., 138., 170., 159., 271., 213., 211., 284., 300.,
       221., 290., 198., 141., 185., 206., 190., 223., 151., 247., 153.,
       155., 297., 253., 208., 140., 227., 183., 22

In [215]:
# TO VIEW VOLUME IDS FOR SPECIFIC STRUCTURE (EACH PREP_ID AND BRAIN STRUCTURE STORED SEPARATELY)
extracted_columns_MD594_SC.volume_id

id
242321    9a661f4f149f6571e0873cf2182d12f5a54a2452
242322    9a661f4f149f6571e0873cf2182d12f5a54a2452
242323    9a661f4f149f6571e0873cf2182d12f5a54a2452
242324    9a661f4f149f6571e0873cf2182d12f5a54a2452
242325    9a661f4f149f6571e0873cf2182d12f5a54a2452
                            ...                   
264258    9a661f4f149f6571e0873cf2182d12f5a54a2452
264259    9a661f4f149f6571e0873cf2182d12f5a54a2452
264260    9a661f4f149f6571e0873cf2182d12f5a54a2452
264261    9a661f4f149f6571e0873cf2182d12f5a54a2452
264262    9a661f4f149f6571e0873cf2182d12f5a54a2452
Name: volume_id, Length: 21942, dtype: object